In [ ]:
import pickle
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import math
from statistics import mean
from matplotlib import cm
from numpy import linalg as LA
import pickle5 as pickle

import warnings
warnings.simplefilter("ignore")

### Velocity for single clusters

In [ ]:

fname = os.path.join('../../../cellmodeller/data/killifish/Sims 11-05_long/Wc__0_75__psi__2__D__1__ftax__0__500__cells_sphere__25__forg__0-20-11-05-12-14', 'step-%05d.pickle')
#fname = os.path.join("/Users/emysk/Desktop/ipre3/killifish/data/Wc__1__psi__0_5__D__0_5__ftax__0__500__cells_sphere__15__forg__0-20-10-22-16-43", 'step-%05d.pickle')

single_clusters_time_step = []

#finding the time step where one cluster forms
for t in range(0,10000,10):
    # Get the cell states
    data = pickle.load(open(fname%t, 'rb'))
    cs = data['cellStates']
        # Construct a graph with cells as nodes and cell-cell contacts as edges
    G = nx.Graph()
    for id,cell in cs.items():
        for n in cell.neighbours:
            G.add_edge(id, n)
        
    for c in nx.connected_components(G):
        if len(c) == 500:
            single_clusters_time_step.append(t)


# mean position for all cells in each time step
all_mean_pos = []
for t in range(0,10000,10):
    for time in single_clusters_time_step:
        if t == time:
            data = pickle.load(open(fname%t, 'rb'))
            cs = data['cellStates']
            cell_pos = []
            for id,state in cs.items():
                cell_pos.append(state.pos)
            cell_array_pos = np.array(cell_pos)
            all_mean_pos.append(cell_array_pos.mean(axis=0))
            
#velocity of cluster between each time step
velocities = []
for p in range(1, len(all_mean_pos)):
    vector = all_mean_pos[p] - all_mean_pos[p - 1]
    velocity = LA.norm(vector)
    #print(velocity)
    velocities.append(velocity)